# Food Spoilage Kernel Case Study

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from matplotlib.lines import Line2D

from lifetime_modeling import *

In [2]:
load_execution_time_from_csv("timing-ablation.csv")
load_embodied_values_from_csv("embodied-carbon-ablation.csv")
load_memory_from_csv("memory-ablation.csv")

workload_accuracies = {
    "DT-Small": 414 / 555,
    "DT-Medium": 465 / 555,
    "DT-Large": 528 / 555,
    "KNN-Large": 549 / 555,
    "KNN-Medium": 547 / 555,
    "KNN-Small": 535 / 555,
    "LR": 545 / 555,
    "MLP": 507 / 555
}


## Lifetime Modeling

In [3]:
workload_dropdown = widgets.Dropdown(
    options=execution_time.keys(),
    value="DT-Small",
    description="Kernel:",
)

lifetime_slider = widgets.FloatSlider(
    value=1.0,
    min=MIN_LIFETIME,
    max=MAX_LIFETIME,
    step=0.5,
    description='Lifetime (yrs)'
)

inf_freq_slider = widgets.BoundedFloatText(
    value=1.0,
    min=MIN_INF_FREQ,
    max=MAX_INF_FREQ,
    step=1.0,
    description='Number of Executed Tasks per Year'
)

power_source_dropdown = widgets.Dropdown(
    options=carbon_intensity_values.keys(),
    value=list(carbon_intensity_values.keys())[0],
    description="Power Source:",
)

carbon_intensity_box = widgets.Text(
    value=carbon_intensity_values[power_source_dropdown.value],
    description="Carbon Int. (CO₂/unit energy)",
)

carbon_intensity_box.disabled = True
def update_carbon_intensity(*args):
    if power_source_dropdown.value == "Custom":
        carbon_intensity_box.disabled = False
    else:
        carbon_intensity_box.value = carbon_intensity_values[power_source_dropdown.value]
        carbon_intensity_box.disabled = True

power_source_dropdown.observe(update_carbon_intensity, 'value')

core_freq_box = widgets.BoundedFloatText(
    value=10000.0,
    min=1000.0,
    max=100000.0,
    step=100.0,
    description='Core Freq'
)

ui = widgets.VBox([
    workload_dropdown,
    lifetime_slider,
    inf_freq_slider,
    power_source_dropdown,
    carbon_intensity_box,
    core_freq_box
])

out = widgets.interactive_output(
    plot_carbon_vs_lifetime,
    {
        'workload_choice': workload_dropdown,
        'lifetime_yrs': lifetime_slider,
        'inf_freq': inf_freq_slider,
        'carbon_intensity': carbon_intensity_box,
        'core_freq': core_freq_box,
    }
)

display(ui, out)

Output()

## Carbon-Accuracy Kernel Modeling

In [4]:
def plot_accuracy_vs_carbon(lifetime_years, tasks_per_year, carbon_intensity):
    x_acc = []
    y_cf = []
    labels = []
    colors = []
    carbon_intensity = float(carbon_intensity) / 3600 / 1000000
    variant_color_map = {
        'Serv': 'tab:blue',
        'Qerv': 'tab:orange',
        'Herv': 'tab:green'
    }
    x_acc = []
    y_cf = []
    labels = []
    colors = []
    markers = []
    sizes = []

    STAR_SIZE = 100
    X_SIZE = 30

    for workload, acc in workload_accuracies.items():
        footprints = []
        for variant in ['Serv', 'Qerv', 'Herv']:
            footprint = compute_total_carbon(variant, workload, lifetime_years, tasks_per_year, carbon_intensity, 10000)[2]
            footprints.append((variant, footprint))
        # Find the variant with the smallest footprint
        min_footprint = min(footprints, key=lambda x: x[1])[1]
        footprints.sort(key=lambda x: x[1], reverse=True)
        for variant, footprint in footprints:
            x_acc.append(acc)
            y_cf.append(footprint)
            colors.append(variant_color_map[variant])
            if footprint == min_footprint:
                labels.append(f"{workload}")
            else:
                labels.append("")

            # Hardcoded - we know that MLP and KNN-Small are only workloads with non-pareto
            if footprint != min_footprint or workload in ["MLP", "KNN-Small"]:
                markers.append('x')
                sizes.append(X_SIZE)
            else:
                markers.append('*')
                sizes.append(STAR_SIZE)

    plt.figure(figsize=(10, 6))
    plt.yscale('log')
    # Plot each point with its marker and size
    for x, y, c, m, s in zip(x_acc, y_cf, colors, markers, sizes):
        plt.scatter(x, y, c=c, s=s, marker=m, alpha=0.9)

    # Place labels to the bottom right of the datapoint
    for i, label in enumerate(labels):
        if label:  # Only annotate non-empty labels
            # xytext=(+8, -8): right and down from the point
            plt.annotate(
                label,
                (x_acc[i], y_cf[i]),
                textcoords="offset points",
                xytext=(0, -8),
                ha='center',
                va='top',
                fontsize=9
            )

    # Custom legend for the variants and marker types
    legend_elements = [
        Line2D([0], [0], marker='*', color='w', label='Pareto-Optimal', markerfacecolor='gray', markeredgecolor='k', markersize=14),
        Line2D([0], [0], marker='x', color='w', label='Not Pareto-Optimal', markerfacecolor='gray', markeredgecolor='k', markersize=8),
        Line2D([0], [0], marker='o', color='w', label='Serv', markerfacecolor='tab:blue', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Qerv', markerfacecolor='tab:orange', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Herv', markerfacecolor='tab:green', markersize=10)
    ]
    plt.legend(handles=legend_elements, title="Variant / Marker")

    plt.xlabel("Accuracy")
    plt.ylabel(f"Total Carbon Footprint over {lifetime_years:.1f} years (gCO2e)")
    plt.title("Accuracy vs. Carbon Footprint")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:

# Reasonable slider ranges for interactive exploration
interact(
    plot_accuracy_vs_carbon,
    lifetime_years=FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Lifetime (years)'),
    tasks_per_year=IntSlider(value=365, min=1, max=10000, step=1, description='Tasks/year'),
    carbon_intensity=Dropdown(
        options=carbon_intensity_values,
        value=carbon_intensity_values["US Avg (2023)"],
        description='Carbon Intensity\n(gCO2e/Wh)'
    )
)


interactive(children=(FloatSlider(value=1.0, description='Lifetime (years)', max=10.0, min=0.1), IntSlider(val…

<function __main__.plot_accuracy_vs_carbon(lifetime_years, tasks_per_year, carbon_intensity)>